In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

try:
    import scikeras
except ImportError:
    !python -m pip install scikeras

from scikeras.wrappers import KerasClassifier, KerasRegressor

from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

### Carregando o conjunto de dados para regressão.

+ Vamos usar o conjunto de dados habitacional da Califórnia e criar um regressor com uma rede neural.

+ Depois de carregar os dados, dividimos em um conjunto de treinamento, um conjunto de validação e um conjunto de teste, e padronizamos todos os atributos.

In [ ]:
# Baixa a base de dados.
housing = fetch_california_housing()

# Divide o conjunto total de exemplos em conjuntos de treinamento e teste.
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)

# Divide o conjunto de treinamento em conjuntos de treinamento (menor) e validação.
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

# Aplica padronização às matrizes de atributos.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Criando o modelo a ser otimizado

Para realizar a otimização com as classes `GridSearchCV` ou `RandomizedSearchCV` da biblioteca SciKit-Learn, **precisamos encapsular nossos modelos em objetos que emulem regressores (ou classificadores) da biblioteca Scikit-Learn**.


O primeiro passo é **criar uma função que irá construir e compilar um modelo do Tensorflow**.


Esta função cria um modelo com a API `Sequencial` para regressão, com a dimensão da entrada e o número de camadas ocultas e neurônios, o passo de aprendizagem, as dimensões e o compila usando um otimizador `SGD` configurado com a taxa de aprendizado fornecida.

In [ ]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    # Instantiate a Sequential object.
    model = keras.models.Sequential()

    # Declare the model
    model.add(keras.layers.Input(shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))

    # Define the optimizer and complile the model.
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

### Regressor Keras

Em seguida, criamos um objeto da classe `KerasRegressor` baseado na função `build_model()`.


O objeto da classe `KerasRegressor` é um **wrapper** (i.e., embrulho) em torno do modelo construído usando a função `build_model()`.


**OBS**.: Passamos para o construtor da classe `KerasRegressor` a função e os parâmetros, definidos na função, que iremos alterar durante o ajuste dos hiperparâmetros.

In [ ]:
keras_reg = KerasRegressor(build_model, n_hidden=1, n_neurons=30, learning_rate=3e-3)

Agora podemos usar esse objeto como um regressor do Scikit-Learn:
+ podemos treiná-lo usando seu método `fit()`,
+ avaliá-lo usando seu método `score()`
+ e usá-lo para fazer predições usando seu método `predict()`.

In [ ]:
keras_reg.fit(X_train, y_train,
              epochs=100,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)]
             )

Epoch 1/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.3469 - val_loss: 36.8626
Epoch 2/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8251 - val_loss: 5.1693
Epoch 3/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.6320 - val_loss: 0.6407
Epoch 4/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5434 - val_loss: 0.4685
Epoch 5/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.5028 - val_loss: 0.4364
Epoch 6/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4779 - val_loss: 0.4193
Epoch 7/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.4616 - val_loss: 0.4084
Epoch 8/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.4502 - val_loss: 0.4014
Epoch 9/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.4414 - val_loss: 0.3962
Epoch 10/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4346 - val_loss: 0.3921
Epoch 11/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4291 - val_loss: 0.3885
Epoch 12/100
363/363 ━━━━━━━━━━━━━━━━━━━

KerasRegressor(
	model=<function build_model at 0x78d32cf06710>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	n_hidden=1
	n_neurons=30
	learning_rate=0.003
)

OBS.: O **score** é o MSE.

In [ ]:
mse_test = keras_reg.score(X_test, y_test)

print('MSE test:', mse_test)

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
MSE test: 0.7438761929100639


In [ ]:
X_new = X_test[:3]

y_pred = keras_reg.predict(X_new)

for i in range(len(y_pred)):
    print('Actual: %1.3f - Predicted: %1.3f' % (y_test[i], y_pred[i]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Actual: 0.477 - Predicted: 0.570
Actual: 0.458 - Predicted: 1.630
Actual: 5.000 - Predicted: 4.196


### Otimização hiperparamétrica

Agora vamos treinar vários modelos com diferentes valores para os hiperparâmetros e ver qual conjunto de valores tem o melhor desempenho no conjunto de validação.


Como existem muitos hiperparâmetros a serem testados, é preferível usar uma pesquisa aleatória (`RandomSearchCV`) em vez de uma pesquisa em grade (`GridSearch`).


Vamos explorar o **número de camadas ocultas**, o **número de neurônios** e o **passo de aprendizado**.

**OBS**.:

+ Listas de valores passados para os hiperparâmetros são amostradas uniformemente.
+ Ao invés de listas, pode-se passar distribuições, de onde valores são amostrados (e.g., a distribuição `reciprocal`).
+ O parâmetro `n_iter` define o número de configurações (i.e., combinações) de hiperparâmetros que são amostradas.

In [9]:
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, scoring='neg_mean_squared_error', cv=3, verbose=2)

rnd_search_cv.fit(X_train, y_train,
                  epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)]
                 )

A saída de streaming foi truncada nas últimas 5000 linhas.
242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4435 - val_loss: 0.4300
Epoch 26/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4385 - val_loss: 0.4250
Epoch 27/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4339 - val_loss: 0.4203
Epoch 28/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4296 - val_loss: 0.4160
Epoch 29/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4255 - val_loss: 0.4119
Epoch 30/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4218 - val_loss: 0.4082
Epoch 31/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4183 - val_loss: 0.4047
Epoch 32/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4150 - val_loss: 0.4015
Epoch 33/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4119 - val_loss: 0.3984
Epoch 34/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4089 - val_loss: 0.3957
Epoch 35/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4062 - val

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(learning_rate=0.003, model=<function build_model at 0x78d32cf06710>, n_hidden=1, n_neurons=30),
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x78d3c4009360>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])},
                   scoring='neg_mean_squared_error', verbose=2)

A busca pode durar muitas horas dependendo do hardware, do tamanho do conjunto de dados, da complexidade do modelo e do valor de `n_iter` e `cv`.

Quando terminar, podemos acessar

+ os melhores hiperparâmetros encontrados,
+ a melhor pontuação
+ e o melhor modelo treinado

In [10]:
# Melhores hiperparâmetros.
rnd_search_cv.best_params_

{'learning_rate': 0.003362564125268811, 'n_hidden': 2, 'n_neurons': 42}

In [11]:
# Score do melhor conjunto de hiperparâmetros.
rnd_search_cv.best_score_

-0.3172083845125933

In [12]:
# Melhor modelo.
model = rnd_search_cv.best_estimator_.model

#### Conclusões

+ A pesquisa aleatória funciona bem (encontra o melhor conjunto de hiperparâmetros) para muitos problemas simples.


+ No entanto, quando temos problemas mais complexos, com conjuntos de dados maiores, essa abordagem explorará apenas uma pequena parte do espaço de hiperparâmetros e pode não encontrar o melhor conjunto de valores.


+ Podemos aliviar parcialmente esse problema auxiliando o processo de pesquisa manualmente:
    + Primeiro executamos uma **pesquisa aleatória rápida usando grandes intervalos de valores de hiperparâmetros**,
    + Na sequência, executamos outra pesquisa usando **intervalos menores, centrados nos melhores valores encontrados** durante a primeira execução e assim por diante.


+ Esperamos que isso ajude a encontrar um bom conjunto de hiperparâmetros. No entanto, isso consome muito tempo.


+ Felizmente, existem outras formas mais eficientes (i.e., inteligentes) de se realizar a otimização dos hiperparâmetros.